## 6.4 Tasky

W tej aplikacji utworzymy prostą listę pogrupowanych zadań - po raz kolejny wykorzystamy `RecyclerView` w dwoma obiektami `ViewHolder`. Całą listę będziemy zapisywać i wczytywać wz pliku za pomocą `SharedPreferences` - do zapisu posłużymy się formatrem `JSON` i biblioteką do serializacji `Gson`. Aplikacja będzie posiadała tylko jeden `Fragment` na którym umieścimy listę i prosty formularz umożliwiający dodawanie kolejnych wpisów.

<table><tr><td><img src="https://media4.giphy.com/media/Dv16lqPAAbY3wcR5yQ/giphy.gif?cid=790b76110f283c94c4653f3110e6fecb2a9d0820a3829b21&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media4.giphy.com/media/NTntAP53dVMiI7x2jO/giphy.gif?cid=790b76112a65996659d22cdbc3f09ca05eafa714f38f57d0&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media2.giphy.com/media/V1hvzLlvSPlEzKhrjM/giphy.gif?cid=790b7611f83aa25dbb5b41859105e74c7fec193a551e4e61&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

### **layout**

Rozpocznijmy od layoutu aktywności do którego dodamy `Fragment`

In [ ]:
<androidx.fragment.app.FragmentContainerView android:id="@+id/fragment_main"
    android:name="pl.udu.uwr.pum.taskykotlin.TaskyFragment"
    xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent" />

oraz layoutu samego fragmentu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".TaskyFragment">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:layout_margin="4dp"
            android:orientation="horizontal">

            <com.google.android.material.textfield.TextInputLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="zadanie">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/taskEditText"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:fontFamily="sans-serif"
                    android:textSize="15sp" />
            </com.google.android.material.textfield.TextInputLayout>
        </LinearLayout>

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_margin="4dp"
            android:layout_height="wrap_content"
            android:orientation="horizontal">

            <com.google.android.material.textfield.TextInputLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="kategoria">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/groupEditText"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:fontFamily="sans-serif"
                    android:textSize="15sp" />
            </com.google.android.material.textfield.TextInputLayout>
        </LinearLayout>

        <Button
            android:id="@+id/addButton"
            android:layout_width="wrap_content"
            android:layout_gravity="center"
            android:text="dodaj"
            android:layout_margin="4dp"
            android:layout_height="wrap_content"/>
    </LinearLayout>

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/rvTasky"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_margin="4dp"/>

    <Button
        android:id="@+id/clearButton"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_marginBottom="8dp"
        android:text="czyść"
        android:layout_height="wrap_content"/>

</LinearLayout>

Dodajmy również layouty dla grupy oraz pojedynczego zadania

`item_group_row.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="horizontal">

    <TextView
        android:id="@+id/groupTextView"
        android:layout_width="0dp"
        android:layout_weight="1"
        android:layout_height="wrap_content"
        android:layout_margin="4dp"
        android:text="Group"
        android:layout_marginStart="8dp"
        android:textSize="24sp" />

</LinearLayout>

`item_task_row.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/taskTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:layout_constraintEnd_toEndOf="parent"
        android:layout_margin="4dp"
        android:layout_marginStart="8dp"
        android:textSize="18sp"
        android:paddingStart="36dp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />


</androidx.constraintlayout.widget.ConstraintLayout>

### **model**

Zdefiniujmy model danych, będziemy posiadać dwie klasy
- `Task` - dla pojedynczego zadania
- `TaskGroup` - dla nagłówka

Na potrzeby `RecyclerView` zdefiniujemy jeszcze trzy typy
- `TaskRow` - reprezentujący rząd
- `Header` - reprezentujący nagłówek
- `Task` - reprezentujący pojedyncze zadanie

In [ ]:
public class TaskRow {
    public static final int TYPE_HEADER = 0;
    public static final int TYPE_TASK = 1;

    private final int rowType;

    public TaskRow(int rowType){
        this.rowType = rowType;
    }

    public int getRowType() {
        return rowType;
    }

    public static class Task extends TaskRow{
        private final String name;

        public Task(String name){
            super(TaskRow.TYPE_TASK);
            this.name = name;
        }

        public String getName() {
            return name;
        }

        @Override
        public boolean equals(Object o) {
            if (this == o) return true;
            if (o == null || getClass() != o.getClass()) return false;
            Task task = (Task) o;
            return name.equals(task.name);
        }

        @Override
        public int hashCode() {
            return Objects.hash(name);
        }
    }

    public static class Header extends TaskRow{
        private final String name;

        public Header(String name){
            super(TaskRow.TYPE_HEADER);
            this.name = name;
        }

        @Override
        public boolean equals(Object o) {
            if (this == o) return true;
            if (o == null || getClass() != o.getClass()) return false;
            Header header = (Header) o;
            return name.equals(header.name);
        }

        @Override
        public int hashCode() {
            return Objects.hash(name);
        }

        public String getName() {
            return name;
        }
    }
}

public class Task {
    private final String name;
    private final TaskGroup type;

    public Task(String name, TaskGroup type){
        this.name = name;
        this.type = type;
    }

    public String getName() {
        return name;
    }

    public TaskGroup getType() {
        return type;
    }
}

public class TaskGroup {
    private final String name;

    public TaskGroup(String name){
        this.name = name;
    }

    public String getName() {
        return name;
    }
}

### **Adapter**

Przejdźmy do `TaskAdapter`

In [ ]:
public class TaskAdapter 
    extends RecyclerView.Adapter<RecyclerView.ViewHolder> {
    
    private final ArrayList<Task> taskList;
    private final ArrayList<TaskRow> rows;

    public TaskAdapter(ArrayList<Task> taskList){
        this.taskList = taskList;
        this.rows = getGroupedTasks();
    }
}

Zadania będziemy przechowywać w liście, w pierwszym kroku musimy pogrupować całą listę

In [ ]:
private ArrayList<TaskRow> getGroupedTasks(){
    ArrayList<TaskRow> list = new ArrayList<>();

    Map<String, List<Task>> grouped = taskList.stream()
        .collect(Collectors.groupingBy(i -> i.getType().getName()));

    for (String group : grouped.keySet()) {
        list.add(new TaskRow.Header(group));
        List<Task> groupingItems = grouped.get(group);
        if (groupingItems != null) {
            groupingItems.forEach(t -> list.add(new TaskRow.Task(t.getName())));
        }
    }

    return list;
}

W pierwszym kroku grupujemy wszystkie zadania według nazwy typu, następnie przechodzimy przez wszystkie elementy i dodajemy nagłówek, oraz listę wszystkich zadań z tym nagłówkiem. Na końcu zwracamy jako listę.

Kolejnym krokiem będzie zdefiniowanie dwóch `ViewHolder` dla nagłówka i zadania.

In [ ]:
private static class TaskViewHolder extends RecyclerView.ViewHolder{

    private final ItemTaskRowBinding binding;

    public TaskViewHolder(ItemTaskRowBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(TaskRow.Task item){
        binding.taskTextView.setText(item.getName());
    }
}

private static class HeaderViewHolder extends RecyclerView.ViewHolder{

    private final ItemGroupRowBinding binding;

    public HeaderViewHolder(ItemGroupRowBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(TaskRow.Header item){
        binding.groupTextView.setText(item.getName());
    }
}

Przejdźmy do metod `RecyclerView`. W `onCreateViewHolder` musimy zwrócić odpowiedni `ViewHolder`

In [ ]:
@NonNull
@Override
public RecyclerView.ViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
    RecyclerView.ViewHolder viewHolder;
    switch (viewType){
        case TaskRow.TYPE_TASK:
            viewHolder = new TaskViewHolder(ItemTaskRowBinding.inflate(
                    LayoutInflater.from(parent.getContext()), parent, false));
            break;
        case TaskRow.TYPE_HEADER:
            viewHolder = new HeaderViewHolder(ItemGroupRowBinding.inflate(
                    LayoutInflater.from(parent.getContext()), parent, false
            ));
            break;
        default:
            throw new IllegalStateException("Unexpected value: " + viewType);
    }

    return viewHolder;
}

Aby to zrobić musimy nadpisać metodę `getItemViewType`

In [ ]:
public int getItemViewType(int position) {
    return rows.get(position).getRowType();
}

`getItemCount` jest rozmiarem naszej pogrupowanej listy

In [ ]:
@Override
public int getItemCount() {
    return rows.size();
}

Na koniec dodajemy implementację metody `onBindViewHolder`.

In [ ]:
@Override
public void onBindViewHolder(@NonNull RecyclerView.ViewHolder holder, int position) {
    TaskRow row = rows.get(position);
    if (row instanceof TaskRow.Task)
        ((TaskViewHolder) holder).bind((TaskRow.Task) row);
    else
        ((HeaderViewHolder) holder).bind((TaskRow.Header) row);
}

Przejdźmy do fragmentu i dodajmy `RecyclerView`

In [ ]:
public class TaskyFragment extends Fragment {

    private FragmentTaskyBinding binding;

    @Nullable
    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, 
                             @Nullable ViewGroup container, 
                             @Nullable Bundle savedInstanceState) {
        binding = FragmentTaskyBinding.inflate(inflater, container, false);
        return binding.getRoot();
    }

    @Override
    public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);

        ArrayList<Task> tasks = SharedPrefUtil.getTaskList(requireContext());

        binding.rvTasky.setLayoutManager(new LinearLayoutManager(requireContext()));
        binding.rvTasky.setAdapter(new TaskAdapter(tasks));
    }
}

W pliku `DataProvider` dodajmy dane testowe

In [ ]:
final public class DataProvider {
    private DataProvider(){}

    public static ArrayList<Task> getTasks(){
        ArrayList<Task> tasks = new ArrayList<>();
            tasks.add(new Task("task1", new TaskGroup("Group 1")));
            tasks.add(new Task("task2", new TaskGroup("Group 1")));
            tasks.add(new Task("task3", new TaskGroup("Group 1")));
            tasks.add(new Task("task4", new TaskGroup("Group 1")));
            tasks.add(new Task("task5", new TaskGroup("Group 1")));
            tasks.add(new Task("task a", new TaskGroup("Group 2")));
            tasks.add(new Task("task b", new TaskGroup("Group 2")));
            tasks.add(new Task("task c", new TaskGroup("Group 2")));
            tasks.add(new Task("task d", new TaskGroup("Group 2")));
            return tasks;
    }
}

Możemy przetestować aplikację

<img src="https://media3.giphy.com/media/qoAHLzTQ0PezrKEhWH/giphy.gif?cid=790b7611c96fcee70976ddb748e72df1f3c57af60cc984ef&rid=giphy.gif&ct=g" width="150" />

Wróćmy do adaptera i napiszmy metodę umożliwiającą dodanie elementu. Chcemy dodać zadanie do właściwej grupy, na koniec wszystkich zadań. Potrzebujemy metodę zwracającą listę wszystkich zadań przynależących do danej grupy

In [ ]:
private List<TaskRow> subList(String groupName){
    return taskList.stream()
            .filter(i -> i.getType().getName().equals(groupName))
            .map(i -> new TaskRow.Task(i.getName()))
            .collect(Collectors.toList());
}

Następnie dodajmy metodę `add`

In [ ]:
public void add(Task task, Context context){}

W pierwszym kroku dodajmy element na listę zadań

In [ ]:
tasksList.add(task);

Znajdźmy nagłówek zadania (jeżeli istnieje)

In [ ]:
TaskRow.Header header = (TaskRow.Header)rows.stream()
    .filter(i -> (i instanceof TaskRow.Header) && ((TaskRow.Header) i)
            .getName()
            .equals(task.getType().getName()))
    .findAny()
    .orElse(null);

Jeżeli nagłówek nie istnieje tworzymy go i dodajemy na listę, następnie dodajemy samo zadanie i wysyłamy odpowiednie powiadomienie do adaptera

In [ ]:
if (header == null){
    rows.add(new TaskRow.Header(task.getType().getName()));
    rows.add(new TaskRow.Task(task.getName()));
    notifyItemInserted(rows.size());

Jeżeli nagłówek istnieje chcemy dodać zadanie na koniec podlisty

In [ ]:
} else {
    int i = rows.indexOf(new TaskRow.Header(task.getType().getName()));
    rows.add(i + subList(task.getType().getName()).size(), new TaskRow.Task(task.getName()));
    notifyItemInserted(i + subList(task.getType().getName()).size());
}

W naszym fragmencie obsłużmy `onClick` przycisku odpowiadającego za dodanie nowego zadania.

In [ ]:
binding.addButton.setOnClickListener(v -> {
    String task = Objects.requireNonNull(
        binding.taskEditText.getText()).toString();
    String cat = Objects.requireNonNull(
        binding.groupEditText.getText()).toString();

    if (!task.isEmpty() && !cat.isEmpty()){
        TaskAdapter adapter = 
            (TaskAdapter) Objects.requireNonNull(binding.rvTasky.getAdapter());
        adapter.add(new Task(task, new TaskGroup(cat)), requireContext());
        binding.groupEditText.getText().clear();
        binding.taskEditText.getText().clear();
    }
});

Dodajmy również obsługę przycisku czyszczącego całą listę

In [ ]:
binding.clearButton.setOnClickListener(v -> {
    TaskAdapter adapter = (TaskAdapter) Objects.requireNonNull(binding.rvTasky.getAdapter());
    adapter.clear(requireContext());
});

Oraz zaimplementujmy metodę `clear` w klasie `TaskAdapter`

In [ ]:
public void clear(Context context){
    taskList.clear();
    notifyItemRangeRemoved(0, rows.size());
    rows.clear();
}

Możemy przetestować aplikację

<img src="https://media0.giphy.com/media/sJcy5w1ByfcZ7g50LC/giphy.gif?cid=790b761149c8246e2b3f29cede919b77aff2aff10a7990c3&rid=giphy.gif&ct=g" width="150" />

### **SharedPreferences -- JSON**

Ostatnim krokiem będzie zapisanie listy zadań do pliku. W tym celu utwórzmy plik `SharedPrefUtil` i dodajmy odpowiednie elementy. Będziemy potrzebować dwie stałe
- nazwę pliku
- klucz listy

In [ ]:
final public class SharedPrefUtil {
    private SharedPrefUtil(){}

    public static final String TASK_LIST = "tasks";
    public static final String TASK_FILE = "task_file_java";
}

Chcemy skorzystać z formatu `json`, więc wykorzystamy bibliotekę `Gson` aby przygotować odpowiednio `String` do zapisu i odczytu. Musimy dodać tą bibliotekę do zależności w pliku `gradle`

In [ ]:
implementation 'com.google.code.gson:gson:2.9.0'

Następnie zaimplementujmy metodę pozwalającą zapisać dane

In [ ]:
public static void saveTaskList(Context context, List<Task> tasks){}

W pierwszym kroku utwórzmy odpowiednio sformatowany `String` za pomocą biblioteki `Gson`

In [ ]:
Gson gson = new Gson();
String json = gson.toJson(tasks);

Następnie dodajmy ten `String` do `SharedPreferences` powiązanego z aplikacją

In [ ]:
SharedPreferences preferences = 
    context.getSharedPreferences(TASK_FILE, Context.MODE_PRIVATE);
preferences.edit().putString(TASK_LIST, json).apply();

Druga metoda pozwoli odczytać zapisany plik.

In [ ]:
public static ArrayList<Task> getTaskList(Context context){}

Po kluczu rozpakowujemy `String` z obiektu `SharedPreferences`

In [ ]:
Gson gson = new Gson();
SharedPreferences preferences = 
    context.getSharedPreferences(TASK_FILE, Context.MODE_PRIVATE);
String json = preferences.getString(TASK_LIST, "");

Jeżeli dostaniemy `null` metoda zwraca pustą listę

In [ ]:
if (json.isEmpty())
    return new ArrayList<>();

W przeciwnym razie rozpakowujemy listę korzystając z bibllioteki `Gson`. Musimy sprecyzować co dokładnie `Gson` ma przekonwertować na listę - musimy jawnie podać typ. O ile w przypadku pojedynczych obiektów nie jest to problemem (przykładowo `Task.class`), o tyle w przypadku listy obiektów musimy wykorzystać typy generyczne czasu wykonania `TypeToken` (więcej informacji [tutaj](https://helw.net/2017/11/09/runtime-generics-in-an-erasure-world/))

In [ ]:
Type type = new TypeToken<ArrayList<Task>>(){}.getType();
return gson.fromJson(json, type);

Metody wywołujemy w wewnątrz metody `add` i `clear` klasy `TaskAdapter`

In [ ]:
public void add(Task task, Context context){
    taskList.add(task);
    SharedPrefUtil.saveTaskList(context, taskList);
    ...
}

public void clear(Context context){
    taskList.clear();
    notifyItemRangeRemoved(0, rows.size());
    rows.clear();
    SharedPrefUtil.saveTaskList(context, taskList);
}

oraz w `onViewCreated` klsay `TaskyFragment`

In [ ]:
ArrayList<Task> tasks = SharedPrefUtil.getTaskList(requireContext());

Możemy przetestować aplikację.

<table><tr><td><img src="https://media4.giphy.com/media/Dv16lqPAAbY3wcR5yQ/giphy.gif?cid=790b76110f283c94c4653f3110e6fecb2a9d0820a3829b21&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media4.giphy.com/media/NTntAP53dVMiI7x2jO/giphy.gif?cid=790b76112a65996659d22cdbc3f09ca05eafa714f38f57d0&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/V1hvzLlvSPlEzKhrjM/giphy.gif?cid=790b7611f83aa25dbb5b41859105e74c7fec193a551e4e61&rid=giphy.gif&ct=g" width="150" /></td></tr></table>